In [ ]:
from importlib import reload
from pathlib import Path
import os
import numpy as np

from obspy.clients.filesystem.sds import Client as SDSClient
from obspy.clients.fdsn import RoutingClient
from obspy.core import UTCDateTime as UTC, read

import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')

from data_quality_control import sds_db, base, util, analysis
from data_quality_control.analysis import Analyzer

## Define parameters

In [ ]:
# NSLC
nslc_code = "GR.BFO..BHZ"

overlap = 60 #3600
fmin, fmax = (4, 14)
nperseg = 2048
winlen_in_s = 3600
proclen = 24*3600
sampling_rate = 20

outdir = 'output'

sds_root = os.path.abspath('../../sample_sds/')
inventory_routing_type = "eida-routing"

In [ ]:
startdate = UTC("2020-12-24")
enddate = UTC("2021-01-15")

In [ ]:
reload( sds_db)
reload(base)
processor = sds_db.SDSProcessor(
        nslc_code,
        inventory_routing_type, 
        sds_root, outdir=outdir, 
        overlap=overlap, nperseg=nperseg, 
        winlen_seconds=winlen_in_s, 
        proclen_seconds=proclen,
        amplitude_frequencies=(fmin, fmax),
        sampling_rate=sampling_rate)

print(processor)

processor.process(startdate, enddate, force_new_file=True)

In [ ]:
reload(analysis)
reload(base)
#reload(util)
lyza = analysis.Analyzer(outdir, nslc_code,
                            fileunit="year")

lyza.get_data(startdate, enddate)

In [ ]:
lyza.plot_spectrogram();